In [1]:
from pylate import indexes, models, retrieve

/accounts/projects/sewonm/prasann/.conda/envs/scaling2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = models.ColBERT(
    model_name_or_path="../cache/colbert_training/contrastive-Qwen_Qwen¬3-Embedding-0.6B-bs8-e11-lr3e-05-gemini_ntrain_ptest/checkpoint-500",
)

In [6]:
index = indexes.PLAID(
    index_folder="../cache/colbert_indices",
    index_name="indextest",
    override=False,
)

retriever = retrieve.ColBERT(index=index)

In [8]:
documents_ids = ["1", "2", "3"]

documents = [
    "ColBERT’s late-interaction keeps token-level embeddings to deliver cross-encoder-quality ranking at near-bi-encoder speed, enabling fine-grained relevance, robustness across domains, and hardware-friendly scalable search.",

    "PLAID compresses ColBERT token vectors via product quantization to shrink storage by 10×, uses two-stage centroid scoring for sub-200 ms latency, and plugs directly into existing ColBERT pipelines.",

    "PyLate is a library built on top of Sentence Transformers, designed to simplify and optimize fine-tuning, inference, and retrieval with state-of-the-art ColBERT models. It enables easy fine-tuning on both single and multiple GPUs, providing flexibility for various hardware setups. PyLate also streamlines document retrieval and allows you to load a wide range of models, enabling you to construct ColBERT models from most pre-trained language models.",
]

# Encode the documents
documents_embeddings = model.encode(
    documents,
    batch_size=1,
    is_query=False, # Encoding documents
    show_progress_bar=True,
)

# Add the documents ids and embeddings to the PLAID index
index.add_documents(
    documents_ids=documents_ids,
    documents_embeddings=documents_embeddings,
)

Encoding documents (bs=1): 100%|██████████| 3/3 [00:16<00:00,  5.61s/it]


In [ ]:
queries_embeddings = model.encode(
    ["query for document 3", "query for document 1"],
    batch_size=32,
    is_query=True, # Encoding queries
    show_progress_bar=True,
)
scores = retriever.retrieve(
    queries_embeddings=queries_embeddings,
    k=10
)
print(scores)

Encoding queries (bs=32): 100%|██████████| 1/1 [00:00<00:00,  5.13it/s]


[[{'id': '3', 'score': 31.82373046875}, {'id': '2', 'score': 31.822265625}, {'id': '1', 'score': 31.80908203125}], [{'id': '3', 'score': 31.82568359375}, {'id': '2', 'score': 31.8251953125}, {'id': '1', 'score': 31.8125}]]
